# Technical Factor - Rate of Change (ROC)

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        (close - m_lag(close, 12)) / m_lag(close, 12) AS ROC,
        m_avg((close - m_lag(close, 12)) / m_lag(close, 12), 6) AS MAROC,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(MAROC > 0 AND m_lag(MAROC, 1) <= 0, 1, 0) AS TRBY1,
        IF(MAROC < 0 AND m_lag(MAROC, 1) >= 0, 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(ROC > 0 AND (ROC - m_lag(ROC, 1)) > 0, 1, 0) AS MTBY1,
        IF(ROC < 0 AND (ROC - m_lag(ROC, 1)) < 0, 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(ROC < -0.05 AND (ROC - m_lag(ROC, 1)) > 0, 1, 0) AS RVBY1,
        IF(ROC > 0.05 AND (ROC - m_lag(ROC, 1)) < 0, 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(MAROC, 1) <= 0.02 AND MAROC > 0.02, 1, 0) AS BKBY1,
        IF(m_lag(MAROC, 1) >= -0.02 AND MAROC < -0.02, 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        ROC,
        MAROC,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,ROC,MAROC,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-01-27,600519.SH,-0.006240,-0.003604,0,0,0,1,0,0,0,0
1,2025-02-05,600519.SH,-0.022423,-0.005862,0,0,0,1,0,0,0,0
2,2025-02-06,600519.SH,-0.021565,-0.008683,0,0,0,0,0,0,0,0
3,2025-02-07,600519.SH,-0.024788,-0.012930,0,0,0,1,0,0,0,0
4,2025-02-10,600519.SH,-0.027024,-0.017758,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
221,2025-12-25,600519.SH,0.026582,0.006367,0,0,1,0,0,0,0,0
222,2025-12-26,600519.SH,0.025243,0.011158,0,0,0,0,0,0,0,0
223,2025-12-29,600519.SH,0.009833,0.012224,0,0,0,0,0,0,0,0
224,2025-12-30,600519.SH,-0.005114,0.010405,0,0,0,1,0,0,0,0
